In [3]:
# 참조문서 디비에 적재한거 그대로 가져오는 버전
# CSV 파일의 source_id 기준으로 조회하는 버전

import pymysql
import json
import pandas as pd

# DB 설정
USER = "root"
PASS = "1234"
HOST = "127.0.0.1"
PORT = 3306
DB = "any_approval"

# 1. CSV 파일에서 source_id 추출 및 중복 제거
df1 = pd.read_csv('actionType_불일치목록.csv')
df2 = pd.read_csv('actionComment_누락목록.csv')
df3 = pd.read_csv('actionComment_누락목록_추가분.csv')  # 추가

# 두 파일의 source_id를 합치고 중복 제거
source_ids_1 = set(df1['source_id'].dropna().astype(str))
source_ids_2 = set(df2['source_id'].dropna().astype(str))
source_ids_3 = set(df3['source_id'].dropna().astype(str))  # 추가
all_source_ids = source_ids_1 | source_ids_2 | source_ids_3  # 합집합에 추가

print(f"📊 actionType_불일치목록.csv: {len(source_ids_1)}건")
print(f"📊 actionComment_누락목록.csv: {len(source_ids_2)}건")
print(f"📊 actionComment_누락목록_추가분.csv: {len(source_ids_3)}건")  # 추가
print(f"📊 중복 제거 후 총: {len(all_source_ids)}건")

# 2. DB 연결
conn = pymysql.connect(
    host=HOST,
    port=PORT,
    user=USER,
    password=PASS,
    database=DB,
    charset='utf8mb4'
)

def safe_json_loads(text_data):
    """안전하게 JSON 파싱"""
    if text_data is None:
        return []
    if isinstance(text_data, str):
        text_data = text_data.strip()
        if not text_data or text_data == '[]':
            return []
    try:
        return json.loads(text_data)
    except:
        return []

try:
    cursor = conn.cursor(pymysql.cursors.DictCursor)
    
    # source_id 목록으로 IN 절 생성
    source_id_list = "', '".join(all_source_ids)
    
    query = f"""
    SELECT id, source_id, doc_num, `references`, doc_type, title, `attaches`, 
           drafter_name, drafter_position, drafter_dept, drafter_email, drafter_dept_code,
           created_at, created_date, doc_body, doc_status, `referrers`, activities, 
           form_name, is_public, end_year
    FROM documents 
    WHERE source_id IN ('{source_id_list}')
    """
    cursor.execute(query)
    rows = cursor.fetchall()
    
    print(f"📊 DB에서 조회된 문서: {len(rows)}건")
    
    # cmds 파일 생성
    with open('new_documents_수정분.cmds', 'w', encoding='utf-8') as f:
        for idx, row in enumerate(rows):
            references_json = safe_json_loads(row['references'])
            
            doc = {
                "sourceId": row['source_id'],
                "docNum": row['doc_num'].replace(' ', '') if row['doc_num'] else row['doc_num'],
                "references": references_json,
                "docType": row['doc_type'],
                "title": row['title'],
                "attaches": safe_json_loads(row['attaches']),
                "drafter": {
                    "positionName": row['drafter_position'],
                    "deptName": row['drafter_dept'],
                    "name": row['drafter_name'],
                    "emailId": row['drafter_email'],
                    "deptCode": row['drafter_dept_code']
                },
                "createdAt": row['created_at'],
                "docBody": row['doc_body'],
                "docStatus": row['doc_status'],
                "referrers": safe_json_loads(row['referrers']),
                "activities": safe_json_loads(row['activities']),
                "formName": row['form_name'],
                "isPublic": bool(row['is_public'])
            }
            
            f.write(f"addDocument {json.dumps(doc, ensure_ascii=False, separators=(',', ':'))}\n")
    
    print(f"✅ 수정된 문서 총 {len(rows)}개를 new_documents_수정분.cmds 파일로 저장했습니다.")
    
except Exception as e:
    print(f"❌ 에러 발생: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    cursor.close()
    conn.close()

📊 actionType_불일치목록.csv: 37건
📊 actionComment_누락목록.csv: 763건
📊 actionComment_누락목록_추가분.csv: 210건
📊 중복 제거 후 총: 977건
📊 DB에서 조회된 문서: 977건
✅ 수정된 문서 총 977개를 new_documents_수정분.cmds 파일로 저장했습니다.


In [2]:
print("CSV source_id 샘플:")
print(list(all_source_ids)[:5])

CSV source_id 샘플:
['18021747', '2006565', '2005109', '13096631', '21496734']


In [4]:
# 각 단계별로 확인해보세요
df1 = pd.read_csv('actionType_불일치목록.csv')
print(f"df1 전체 행: {len(df1)}")
print(f"df1 source_id 비어있는 행: {df1['source_id'].isna().sum()}")
print(f"df1 source_id 중복: {len(df1) - df1['source_id'].nunique()}")

df1 전체 행: 47
df1 source_id 비어있는 행: 0
df1 source_id 중복: 10


In [6]:
# 각 단계별로 확인해보세요
df1 = pd.read_csv('actionComment_누락목록_추가분.csv')
print(f"df1 전체 행: {len(df1)}")
print(f"df1 source_id 비어있는 행: {df1['source_id'].isna().sum()}")
print(f"df1 source_id 중복: {len(df1) - df1['source_id'].nunique()}")

df1 전체 행: 212
df1 source_id 비어있는 행: 0
df1 source_id 중복: 2


In [5]:
# 각 단계별로 확인해보세요
df1 = pd.read_csv('actionComment_누락목록.csv')
print(f"df1 전체 행: {len(df1)}")
print(f"df1 source_id 비어있는 행: {df1['source_id'].isna().sum()}")
print(f"df1 source_id 중복: {len(df1) - df1['source_id'].nunique()}")

df1 전체 행: 788
df1 source_id 비어있는 행: 0
df1 source_id 중복: 25
